# 4. Advanced RNN Units
We are now going to move into the realm of advanced RNN units. Up until this point, we have been dealing with the simple recurrent unit exclusively. However, implementing **Rated** units, **Gated Recurrent Units**, and **Long Short-Term Memory** will allow our models to become far more powerful.

## 1. Rated RNN Units
A **rated recurrent unit** is simply a straightforward modification to the simple recurrent unit. Recall, a simple recurrent unit has the form:

<img src="https://drive.google.com/uc?id=1tGeRQ2PZsHbFq8XFsj32NC3gQ-46ilWZ" width="320">

Which can be observed on a lower level as:

<img src="https://drive.google.com/uc?id=1F0wDzeyGBAa1l4_mDrIAiejherkAkb8p" width="300">

And mathematically, we can define $h(t)$ as:

$$h(t) = f\big(W_x^T x(t) + W_h^T h(t-1)\big)$$

Where $f$ is our chosen activation function. The idea is that we we want to weight two things:

1. $f\big(x(t), h(t-1)\big)$, which is the output that we would have gotten in a simple recurrent unit.
2. $h(t-1)$, the previous value of the hidden state.

In order to accomplish this weighting, we use a matrix known as the **rate matrix**, $z$, which is the same size as the hidden layer. We then perform an element by element multiplication on each dimension:

$$h(t) = (1-z) \odot h(t-1) + z \odot f\big( x(t), h(t-1)\big)$$

$z$ is known as the rate. You may notice that this looks very similar to the low pass filter that we created in the previous post. The idea here to is to learn $z$ in a way that it let's the some values from previous points in time carry greater weight, and others depend mainly on the most recent values. Visually, the rated recurrent unit looks like:

<img src="https://drive.google.com/uc?id=1Tx7TAd3tUV-7-ezVPbJztx8dZyA3yC7m" width="600">

The changes needed to implement this in code are relatively small and straightforward. We simply need to add a new param of size $M$, and include the above equation in the `recurrence` function.

### 1.1.1 Rate Matrix Modifications
There are more options for the rate matrix than simply what was discussed above. For example, we can make it dependent on the input and previous hidden state via a sigmoid and more weight matrices. 

We can also make it a matrix (size $M x M$) so that it multiplies against $h$ with matrix multiplication, instead of element wise multiplication. In this case, you can picture what is happening as the same type of everything connected to everything scenario that we had for the hidden to hidden state. 

### 1.1.2 Modifications to Poetry Generation
The next thing that we are going to do is redo our poem generation example with the rated recurrent unit. It will be clear that this is just a small modification from the simple recurrent unit, so the code is almost exactly the same. Note, this new parameter will be trained in exactly the same way as all the other parameters-via gradient descent. In fact, when we look at more complex models the training will still remiain the same. 

To add a bit more complexity to the mix, we are going to try and solve the problem of the lines ending too quickly. Recall that this is because we have many training samples that result in the next word being the end token. Because the end token shows up in every line, it is over represented in the training data. To prevent this, we will only train for going to the end of the sequence 10% of the time. Otherwise we stop on the second to last word. 

Another change that we will make is that we will not model the initial distribution anymore. Recall, we use this to prevent every line from starting with the same word. This would happen because neural networks will give us the same prediction every time if we use the argmax. Instead, we will use the `START` token as the input, and the output will represent a probability distribution. This is valid because we already know that softmax gives us a valid probability distribution. We can then sample from this distribution to generate the next word. This way the sequences that we generate will be more stochastic, and it will treat the output probability as an actual probability, rather than a deterministic prediction:

```
p(w0) = softmax(f(START))
w0 = randint(V, p=p(w0))
```

### 1.1.3 Rated RNN Unit in Code

In [11]:
import theano
import theano.tensor as T
import numpy as np
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from rnn_util import init_weight, get_robert_frost


class SimpleRNN:
    def __init__(self, D, M, V):
        self.D = D # dimensionality of word embedding
        self.M = M # hidden layer size
        self.V = V # vocabulary size

    def fit(self, X, learning_rate=10., mu=0.9, reg=0., activation=T.tanh, epochs=500, show_fig=False):
        N = len(X)
        D = self.D
        M = self.M
        V = self.V

        # initial weights
        We = init_weight(V, D)
        Wx = init_weight(D, M)
        Wh = init_weight(M, M)
        bh = np.zeros(M)
        h0 = np.zeros(M)
        # z  = np.ones(M)
        Wxz = init_weight(D, M)
        Whz = init_weight(M, M)
        bz  = np.zeros(M)
        Wo = init_weight(M, V)
        bo = np.zeros(V)

        thX, thY, py_x, prediction = self.set(We, Wx, Wh, bh, h0, Wxz, Whz, bz, Wo, bo, activation)

        lr = T.scalar('lr')

        cost = -T.mean(T.log(py_x[T.arange(thY.shape[0]), thY]))
        grads = T.grad(cost, self.params)
        dparams = [theano.shared(p.get_value()*0) for p in self.params]

        updates = []
        for p, dp, g in zip(self.params, dparams, grads):
            new_dp = mu*dp - lr*g
            updates.append((dp, new_dp))

            new_p = p + new_dp
            updates.append((p, new_p))

        self.predict_op = theano.function(inputs=[thX], outputs=prediction)
        self.train_op = theano.function(
            inputs=[thX, thY, lr],
            outputs=[cost, prediction],
            updates=updates
        )

        costs = []
        for i in range(epochs):
            X = shuffle(X)
            n_correct = 0
            n_total = 0
            cost = 0
            for j in range(N):
                if np.random.random() < 0.1:
                    input_sequence = [0] + X[j]
                    output_sequence = X[j] + [1]
                else:
                    input_sequence = [0] + X[j][:-1]
                    output_sequence = X[j]
                n_total += len(output_sequence)

                # we set 0 to start and 1 to end
                c, p = self.train_op(input_sequence, output_sequence, learning_rate)
                # print "p:", p
                cost += c
                # print "j:", j, "c:", c/len(X[j]+1)
                for pj, xj in zip(p, output_sequence):
                    if pj == xj:
                        n_correct += 1
            if i % 50 == 0:
                print("i:", i, "cost:", cost, "correct rate:", (float(n_correct)/n_total))
            if (i + 1) % 500 == 0:
                learning_rate /= 2
            costs.append(cost)

        if show_fig:
            plt.plot(costs)
            plt.show()
    
    def save(self, filename):
        np.savez(filename, *[p.get_value() for p in self.params])

    @staticmethod
    def load(filename, activation):
        # TODO: would prefer to save activation to file too
        npz = np.load(filename)
        We = npz['arr_0']
        Wx = npz['arr_1']
        Wh = npz['arr_2']
        bh = npz['arr_3']
        h0 = npz['arr_4']
        Wxz = npz['arr_5']
        Whz = npz['arr_6']
        bz = npz['arr_7']
        Wo = npz['arr_8']
        bo = npz['arr_9']
        V, D = We.shape
        _, M = Wx.shape
        rnn = SimpleRNN(D, M, V)
        rnn.set(We, Wx, Wh, bh, h0, Wxz, Whz, bz, Wo, bo, activation)
        return rnn

    def set(self, We, Wx, Wh, bh, h0, Wxz, Whz, bz, Wo, bo, activation):
        self.f = activation

        # redundant - see how you can improve it
        self.We = theano.shared(We)
        self.Wx = theano.shared(Wx)
        self.Wh = theano.shared(Wh)
        self.bh = theano.shared(bh)
        self.h0 = theano.shared(h0)
        self.Wxz = theano.shared(Wxz)
        self.Whz = theano.shared(Whz)
        self.bz = theano.shared(bz)
        self.Wo = theano.shared(Wo)
        self.bo = theano.shared(bo)
        self.params = [self.We, self.Wx, self.Wh, self.bh, self.h0, self.Wxz, self.Whz, self.bz, self.Wo, self.bo]

        thX = T.ivector('X')
        Ei = self.We[thX] # will be a TxD matrix
        thY = T.ivector('Y')

        def recurrence(x_t, h_t1):
            # returns h(t), y(t)
            hhat_t = self.f(x_t.dot(self.Wx) + h_t1.dot(self.Wh) + self.bh)
            z_t = T.nnet.sigmoid(x_t.dot(self.Wxz) + h_t1.dot(self.Whz) + self.bz)
            h_t = (1 - z_t) * h_t1 + z_t * hhat_t
            y_t = T.nnet.softmax(h_t.dot(self.Wo) + self.bo)
            return h_t, y_t

        [h, y], _ = theano.scan(
            fn=recurrence,
            outputs_info=[self.h0, None],
            sequences=Ei,
            n_steps=Ei.shape[0],
        )

        py_x = y[:, 0, :]
        prediction = T.argmax(py_x, axis=1)
        self.predict_op = theano.function(
            inputs=[thX],
            outputs=[py_x, prediction],
            allow_input_downcast=True,
        )
        return thX, thY, py_x, prediction


    def generate(self, word2idx):
        # convert word2idx -> idx2word
        idx2word = {v:k for k,v in word2idx.items()}
        V = len(word2idx)
        n_lines = 0
        X = [ 0 ]
        while n_lines < 4:
            # print "X:", X
            PY_X, _ = self.predict_op(X)
            PY_X = PY_X[-1].flatten()
            P = [ np.random.choice(V, p=PY_X)]
            X = np.concatenate([X, P]) # append to the sequence
            P = P[-1] # just grab the most recent prediction
            if P > 1:
                # it's a real word, not start/end token
                word = idx2word[P]
                print(word, end=" ")
            elif P == 1:
                # end token
                n_lines += 1
                X = [0]
                print('')


def train_poetry():
    sentences, word2idx = get_robert_frost()
    rnn = SimpleRNN(50, 50, len(word2idx))
    rnn.fit(sentences, learning_rate=1e-4, show_fig=True, activation=T.nnet.relu, epochs=2000)
    rnn.save('RRNN_D50_M50_epochs2000_relu.npz')

def generate_poetry():
    sentences, word2idx = get_robert_frost()
    rnn = SimpleRNN.load('RRNN_D50_M50_epochs2000_relu.npz', T.nnet.relu)
    rnn.generate(word2idx)


if __name__ == '__main__':
#     train_poetry()
    generate_poetry()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:139: UserWarning: DEPRECATION: If x is a vector, Softmax will not automatically pad x anymore in next releases. If you need it, please do it manually. The vector case is gonna be supported soon and the output will be a vector.


and give of people a hornyhanded kindness as the arm ground snow snow cellar out 
and piano tell here him was stop in her me 
of then acquaintance out but a house them 
they be leafs a long it didnt held 


## 2. Gated Recurrent Unit
We are now going to introduce a unit that is more powerful than the ellman unit, the **Gated Recurrent Unit**, (GRU). GRU's were introduced in 2014, while LSTM's (which we will be going over next) were introduced in 1997. I have chosen to start with GRU's because they are slightly less complex, and thus a better place to begin. The GRU is very similar to LSTM, incorporating many of the same concepts, but having far fewer parameters, meaning it can train faster at a constant hidden layer size. 

### 2.1 GRU Architecture
To start, let's go over the architecture of the GRU. Before we do that, however, I want to quickly recap the previous architectures that we have seen, particularly treating the hidden unit as a black box of sorts. This compartmental point of view will help us in extending the simple unit and rated unit to the GRU. 

**Feedforward Unit** <br>
In the simplest feedforward network, this black box simply contains some nonlinear function like $tanh$ or $relu$:

<img src="https://drive.google.com/uc?id=16Qe6uVFsX58tPqla11bUDtIuTyqlyyho" width="300">

**Simple Recurrent Unit**<br>
In a simple recurrent network, we just connect the output of the black box back to itself, with a time delay of one:

<img src="https://drive.google.com/uc?id=1lojDOZVNBmCE23wbEmNXgD7PfYWAo_eI" width="300">

**Rated Recurrent Unit**<br>
With the rated recurrent unit, we add a rating operation between what would have been the output of the simple recurrent network, and the previous output value. 

<img src="https://drive.google.com/uc?id=1nLz2HGaYhP0JWnbm2FXIbpThFkb6toaM" width="500">

We can think of of this new operation as a gate. Since it has to take on a value between 0 and 1, and the other gate has to take on the 1 minus that value, it is a gate that is choosing between two things: taking on the old value or taking on the new value. The result here is that we get a mixture of both.

**Gated Recurrent Unit**<br>
Finally, we arrive at the gated recurrent unit! The architecure simply requires that we add one more gate: 

<img src="https://drive.google.com/uc?id=1BqfOZ1LYDmOhXghEa4NJ7VIQbLbArzMY" width="500">

The above diagram corresponds with the following mathematical equations:

$$r_t = \sigma \big(x_t W_{xr} + h_{t-1}W_{hr} + b_r \big)$$

$$z_t = \sigma \big(x_t W_{xz} + h_{t-1}W_{hz} + b_z\big)$$

$$\hat{h}_t = g \big(x_t W_{xh} + (r_t \odot h_{t-1})W_{hh} + b_h\big)$$

$$h_t = (1 - z_t) \odot h_{t-1} + z_t \odot \hat{h}_t$$

Note: $g$ represents an activation, and $\odot$ is the symbol for element wise multiplication. With that said, we are simply seeing more of the same thing that we have been encountering thus far; weight matrices multiplied by inputs and passed through non linear functions and gates. 

Again, we see the update gate $z_t$ that we encountered with the rated unit. It still balances how much of the previous hidden value and how much of the new candidate hidden value combines to get the new hidden value. 

The new component is $r_t$, or the **reset gate**, which has the exact same functional form as the update gate- all of its weights are the same size. However, it is position in the black box is different. The reset gate is multiplied by the previous hidden state value. It controls how much of the previous hidden state we will consider, when we create the new candidate hidden value. In other words, it has the ability to reset the hidden value. 

For instance, consider the situation where $r_t$ is equal to 0, then we get:

$$\hat{h}_t = g \big(x_t W_{xh} + b_h\big)$$

This would be as if $x_t$ were the beginning of a new sequence. Note that this is not the full picture, since $\hat{h}$ is only a candidate for the new $h_t$, since $h_t$ will be a combination of $\hat{h}$ and $h_{t-1}$ (which is controlled by the updated gate $z_t$). 

Now, if that all sounds a bit complex, there is a practical way to reason with it. At the most general level, we are simply adding more parameters to our model, and making it more expressive. Adding more parameters allows us to fit more complex patterns. 

### 2.2 GRU in Code
When it comes to implement a GRU in code, it should be relatively simple if you already understand the simple recurrent unit and the rated recurrent unit. We simply need to add more weights, and modify the recurrence. The next step to making our code better is to modularize it. We have mentioned that the GRU can be looked at as a black box. To do this, we can simply make the GRU into a class so that it can be abstracted away. By doing this, we can stack GRU's, meaning anywhere that a hidden layer could go, a GRU can go as well. This allows us to just think of it as a thing that takes an input and produces an output. The fact that it contains a memory of previous inputs is just an internal detail of the black box. 

Some pseudocode is show below:

```
class GRU:
    def __init__(Mi, Mo):
        Wxr = random(Mi, Mo)
        # ...
    
    def recurrence(x_t, h_t1):
        r = sigmoid(x_t.dot(Wxr) + h_t1.dot(Whr) + br)
        # ...
        return (1 - z)*h_t1 + z*hhat
    
    def output(x):
        return scan(recurrence, x)     
```

And, a full class implementation:

In [1]:
import numpy as np
import theano
import theano.tensor as T

from rnn_util import init_weight


class GRU:
    def __init__(self, Mi, Mo, activation):
        self.Mi = Mi
        self.Mo = Mo
        self.f = activation

        Wxr = init_weight(Mi, Mo) # Input into reset gate
        Whr = init_weight(Mo, Mo) # Hidden into reset gate
        br = np.zeros(Mo)         # Bias reset gate
        Wxz = init_weight(Mi, Mo) # Input to update gate
        Whz = init_weight(Mo, Mo) # Hidden to update gate
        bz = np.zeros(Mo)         # Bias update gate
        Wxh = init_weight(Mi, Mo)
        Whh = init_weight(Mo, Mo)
        bh = np.zeros(Mo)
        h0 = np.zeros(Mo)

        # Create theano variables
        self.Wxr = theano.shared(Wxr)
        self.Whr = theano.shared(Whr)
        self.br = theano.shared(br)
        self.Wxz = theano.shared(Wxz)
        self.Whz = theano.shared(Whz)
        self.bz = theano.shared(bz)
        self.Wxh = theano.shared(Wxh)
        self.Whh = theano.shared(Whh)
        self.bh = theano.shared(bh)
        self.h0 = theano.shared(h0)
        self.params = [self.Wxr, self.Whr, self.br, self.Wxz, self.Whz, self.bz, self.Wxh, self.Whh, self.bh, self.h0]

    def recurrence(self, x_t, h_t1):
        r = T.nnet.sigmoid(x_t.dot(self.Wxr) + h_t1.dot(self.Whr) + self.br) # Reset gate
        z = T.nnet.sigmoid(x_t.dot(self.Wxz) + h_t1.dot(self.Whz) + self.bz) # Update gate
        hhat  = self.f(x_t.dot(self.Wxh) +(r*h_t1).dot(self.Whh) + self.bh)  # Candidate for h
        h = (1 - z)*h_t1 + z*hhat
        return h

    def output(self, x):
        # Output for this unit taking in an input sequence X. X is 2 dimensional: Time x Dimension
        h, _ = theano.scan(
            fn=self.recurrence,
            sequences=x,
            outputs_info=[self.h0],
            n_steps=x.shape[0]
        )
        return h

## 3. Long Short-Term Memory (LSTM)
We are now going to move on from the GRU and start working with the **LSTM**. LSTM stands for **Long Short-Term Memory**, and it is a type of recurrent unit that has become very popular in recent years due to its superior performance/ability to overcome the vanishing gradient problem. Each recurrent unit that we have dug into has gotten progressively more complex, and each incorporated concepts from the previous unit. 

The LSTM is the most complex recurrent unit we will cover. However, that complexity does not involve abstract ideas or new mathematics; rather, the number of components that we are dealing with is higher. Essentially, we will now have **three different gates**, called the **input**, **output**, and **forget** gate. Additionally, we are going to add yet another internal unit that will exist alongside the hidden state, sometimes referred to as a **memory cell**. One way to think of this cell is that it takes the place of $\hat{h}$, or the candidate hidden value, when we talked about the GRU.

### 3.1 LSTM Mathematics
Now, the input gate, $i_t$, and forget gate, $f_t$, should remind you of the rate gate, $z_t$, from the GRU. Before, we were using $z_t$ and $1- z_t$; however, now we just have two seperate gates:

**Input Gate**<br>
The input gate just controls how much of the new value goes into the cell:

$$i_t = \sigma\big(x_t W_{xi} + h_{t-1}W_{hi} + c_{t-1}W_{ci} + b_i\big)$$

**Forget Gate**<br>
The forget gate controls how much of the previous cell value goes into the current cell value:

$$f_t = \sigma\big( x_t W_{xf} + h_{t-1}W_{hf} + c_{t-1}W_{cf} + b_f\big)$$

**Candidate**<br>
The candidate for the new cell value looks a lot like what would be the simple recurrent unit's value, right before it gets multiplied by the input gate:

$$c_t = f_tc_{t-1} + i_t \;tanh\big(x_t W_{xc} + h_{t-1}W_{hc} + b_c\big)$$

**Output Gate**<br>
Finally, the output gate takes into account everything: the input at time $t$, the previous hidden state, and the current cell value:

$$o_t = \sigma \big( x_t W_{xo} + h_{t-1}W_{ho} + c_t W_{co} + b_o\big) $$

**New Hidden State**<br>
The new hidden state is just the $tanh$ of the cell value, multiplied by the output gate:

$$h_t = o_t \; tanh(c_t)$$

### 3.2 LSTM Parameters
Now, we just introduced a substantial number of new parameters, so it may be useful to try and break down exactly what they are. 

|Model Component   |Parameters|Depends on|
|------------------|----------|----------|
|**Input Gate**    |$W_{xi}$, $W_{hi}$, $W_{ci}$, $b_i$|$x_t$, $h_{t-1}$, $c_{t-1}$|
|**Forget Gate**   |$W_{xf}$, $W_{hf}$, $W_{cf}$, $b_f$|$x_t$, $h_{t-1}$, $c_{t-1}$|
|**Candidate Cell**|$W_{xc}$, $W_{hc}$, $b_c$|$x_t$, $h_{t-1}$|
|**Output Gate**   |$W_{xo}$, $W_{ho}$, $W_{co}$, $b_o$|$x_t$, $h_{t-1}$, $c_{t}$|

In total, we have 15 new weights and biases for this black box! We have come a long way from our original feedforward net, where we only had 1 weight and 1 bias. Another way that we can think of this is that as we add more parameters, we enable our model to be more expressive. 

### 3.3 LSTM Architecture
Now, from an architecture perspective, an LSTM has the high level form of: 

<img src="https://drive.google.com/uc?id=1B1OX2ew8J3AEA2mOdjOLtyqSX4jbhk1s" width="700">

Where $\hat{c}$ is just meant to represent $x_t W_{xc} + h_{t-1}W_{hc} + b_c$ in the candidate equation. Now, for some this diagram may be sufficient, however, I would prefer to have a bit more insight to the inner working of the LSTM at a lower level. This can be seen below:

<img src="https://drive.google.com/uc?id=1eaCY0O9FNtwMZyRRHKNgsIcGcqohX06Q" width="700">

<img src="https://drive.google.com/uc?id=1YSdHlemvYVdlQ3ZxgRFsEWS35OOhIAdV" width="300">

And we can highlight the actual cell as follows:

<img src="https://drive.google.com/uc?id=14cDQlcuK8cMXRXSZ2nVG8kvGSI77Qloo" width="700">

I encourage you to take the equations the we defined earlier and follow them through the architecture diagram above to make sure they make sense. I should point out that in order to reduce visual noise in the diagram I didn't inclue the weight matrices or biases.

### 3.4 LSTM in Code
We can now implement an LSTM in code:

In [2]:
import numpy as np
import theano
import theano.tensor as T

from rnn_util import init_weight


class LSTM:
    def __init__(self, Mi, Mo, activation):
        self.Mi = Mi
        self.Mo = Mo
        self.f = activation

        Wxi = init_weight(Mi, Mo)
        Whi = init_weight(Mo, Mo)
        Wci = init_weight(Mo, Mo)
        bi = np.zeros(Mo)
        Wxf = init_weight(Mi, Mo)
        Whf = init_weight(Mo, Mo)
        Wcf = init_weight(Mo, Mo)
        bf = np.zeros(Mo)
        Wxc = init_weight(Mi, Mo)
        Whc = init_weight(Mo, Mo)
        bc = np.zeros(Mo)
        Wxo = init_weight(Mi, Mo)
        Who = init_weight(Mo, Mo)
        Wco = init_weight(Mo, Mo)
        bo = np.zeros(Mo)
        c0 = np.zeros(Mo)
        h0 = np.zeros(Mo)

        # Create Theano variables
        self.Wxi = theano.shared(Wxi)
        self.Whi = theano.shared(Whi)
        self.Wci = theano.shared(Wci)
        self.bi = theano.shared(bi)
        self.Wxf = theano.shared(Wxf)
        self.Whf = theano.shared(Whf)
        self.Wcf = theano.shared(Wcf)
        self.bf = theano.shared(bf)
        self.Wxc = theano.shared(Wxc)
        self.Whc = theano.shared(Whc)
        self.bc = theano.shared(bc)
        self.Wxo = theano.shared(Wxo)
        self.Who = theano.shared(Who)
        self.Wco = theano.shared(Wco)
        self.bo = theano.shared(bo)
        self.c0 = theano.shared(c0)
        self.h0 = theano.shared(h0)
        self.params = [
            self.Wxi,
            self.Whi,
            self.Wci,
            self.bi,
            self.Wxf,
            self.Whf,
            self.Wcf,
            self.bf,
            self.Wxc,
            self.Whc,
            self.bc,
            self.Wxo,
            self.Who,
            self.Wco,
            self.bo,
            self.c0,
            self.h0,
        ]

    def recurrence(self, x_t, h_t1, c_t1):
        i_t = T.nnet.sigmoid(x_t.dot(self.Wxi) + h_t1.dot(self.Whi) + c_t1.dot(self.Wci) + self.bi)
        f_t = T.nnet.sigmoid(x_t.dot(self.Wxf) + h_t1.dot(self.Whf) + c_t1.dot(self.Wcf) + self.bf)
        c_t = f_t*c_t1 + i_t*T.tanh(x_t.dot(self.Wxc) + h_t1.dot(self.Whc) + self.bc)
        o_t = T.nnet.sigmoid(x_t.dot(self.Wxo) + h_t1.dot(self.Who) + c_t.dot(self.Wco) + self.bo)
        h_t = o_t*T.tanh(c_t)
        return h_t, c_t

    def output(self, x):
        # input X should be a matrix (2-D)
        # rows index time
        [h, c], _ = theano.scan(
            fn=self.recurrence,
            inputs=x,
            outputs_info=[self.h0, self.c0], # Include c so that recurrence has access to it!
            n_steps=x.shape[0]
        )
        return h